<a href="https://colab.research.google.com/github/jw00oo1/Dacon/blob/main/11_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import sklearn
import matplotlib.pyplot as plt
import os
import tensorflow.keras.layers as layers
from pathlib import Path

In [ ]:
path = Path(os.getcwd()) / 'data'
if not os.path.isdir(path):
    os.mkdir(path)

In [ ]:
train_csv = pd.read_csv(path / 'train.csv')
test_csv = pd.read_csv(path / 'test_x.csv')
submission_csv = pd.read_csv(path / 'sample_submission.csv')

train = train_csv.drop('index', axis=1)
test = test_csv.drop('index', axis=1)

In [ ]:
from sklearn.model_selection import train_test_split

train_y = train['voted']
train_x = train.drop('voted', axis=1)


In [ ]:
pd.set_option('display.max_rows',len(train))
train.dtypes
#age_group, race, religion, gender

QaA           float64
QaE             int64
QbA           float64
QbE             int64
QcA           float64
QcE             int64
QdA           float64
QdE             int64
QeA           float64
QeE             int64
QfA           float64
QfE             int64
QgA           float64
QgE             int64
QhA           float64
QhE             int64
QiA           float64
QiE             int64
QjA           float64
QjE             int64
QkA           float64
QkE             int64
QlA           float64
QlE             int64
QmA           float64
QmE           float64
QnA           float64
QnE           float64
QoA           float64
QoE           float64
QpA           float64
QpE           float64
QqA           float64
QqE           float64
QrA           float64
QrE           float64
QsA           float64
QsE           float64
QtA           float64
QtE           float64
age_group      object
education     float64
engnat        float64
familysize    float64
gender         object
hand      

In [ ]:
train_x = train_x.to_numpy()
train_y = train_y.to_numpy()
train_x, val_x, train_y, val_y = train_test_split(train_x, train_y, test_size=0.2)

#data처리

In [ ]:
print(train['gender'].isnull().values.any())
print(test['gender'].isnull().values.any())

True
False


In [ ]:
train.shape

(19406, 77)

In [ ]:
print(train.isnull().sum(axis=0))

In [ ]:
#nan이 들어간 열 하나 제거
train = train.dropna()
#nan이 마지막 8개인가 컬 wf 인가 하나씩밖에없음

In [ ]:
train.shape

(19405, 77)

In [ ]:
#pandas to numpy with categorical
def df_to_np(dataframe):
    """pandas to numpy with categorical"""
    dataframe = dataframe.copy()
    np_x = pd.get_dummies(dataframe, columns=['age_group', 'gender', 'race', 'religion'])
    np_y = []
    if 'voted' in np_x:
        np_y = np_x['voted']
        np_x = np_x.drop('voted', axis = 1)
    return np_x, np_y

In [ ]:
train_x, train_y = df_to_np(train)
train_y = train_y - 1

In [ ]:
#정규화
def normalize(dataset):
    norm = (dataset - dataset.mean(axis=0)) / (dataset.std(axis=0))
    return norm

In [ ]:
time = train_x.iloc[:,1]
ans = train_x.iloc[:,0]
for i in range(2,40):
    if i%2:
        time = pd.concat([time, train.iloc[:,i]], axis=1)
    else :
        ans = pd.concat([ans, train.iloc[:,i]],axis=1)

In [ ]:
normalize(time)

In [ ]:
norm_train = pd.concat([train_x.iloc[:,40:], normalize(time), ans], axis=1)

In [ ]:
norm_train.iloc[1,:]

In [ ]:
#not norm
train_x = train_x.iloc[:,40:]

In [ ]:
train_x, val_x, train_y, val_y = train_test_split(train_x, train_y, test_size=0.2)
print(train_x.shape, val_x.shape, train_y.shape, val_y.shape)

(12419, 60) (3105, 60) (12419,) (3105,)


#DNN

In [ ]:
leaky_relu = tf.nn.leaky_relu
model = tf.keras.Sequential([
                             layers.Input(shape=(60,)),
                             layers.Dense(128, activation= 'elu'),
                             layers.Dropout(0.4),
                             layers.Dense(64, activation='elu'),
                             layers.Dropout(0.4),
                             layers.Dense(64, activation='elu'),
                             layers.Dropout(0.4),
                             layers.Dense(32, activation='elu'),
                             layers.Dense(1, activation='sigmoid')

])

model.compile(optimizer = tf.keras.optimizers.Adam(),
              loss='binary_crossentropy',
              metrics=[tf.keras.metrics.AUC()])

In [ ]:
model.fit(train_x, train_y, epochs=100, batch_size=32, validation_data=(val_x, val_y))

#Adaboost

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier

In [ ]:
model = AdaBoostClassifier(n_estimators=5, random_state=42)

In [ ]:
model.fit(train_x, train_y)

AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None, learning_rate=1.0,
                   n_estimators=5, random_state=42)

In [ ]:
print(model.score(train_x, train_y))

0.6762694133741047


#1. tensorflow - managing formed data

In [ ]:
from tensorflow import feature_column
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

In [ ]:
t_train, t_val = train_test_split(train, test_size=0.2)
print(len(t_train), ': 학습 데이터 수')
print(len(t_val) , ': val 데이터 수')

36425 : 학습 데이터 수
9107 : val 데이터 수


In [ ]:
if 'voted' not in test:
    print('no')
else :
    print('yes')

#multi col - set.issubset

no


In [ ]:
#pandas to tf.data
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
    dataframe = dataframe.copy()
    labels = dataframe.pop('voted')
    ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
    if shuffle:
        ds = ds.shuffle(buffer_size=len(dataframe))
    ds = ds.batch(batch_size)
    return ds

In [ ]:
batch_size = 5
train_ds = df_to_dataset(t_train, batch_size=batch_size)
val_ds = df_to_dataset(t_val, shuffle=False, batch_size=batch_size)

26747    2
16616    2
170      2
5331     2
20913    1
        ..
21859    2
1412     2
27984    1
9221     2
13973    1
Name: voted, Length: 36425, dtype: int64
10878    2
25103    2
41150    2
31352    1
40189    1
        ..
1268     2
6550     2
10706    2
35329    2
37965    2
Name: voted, Length: 9107, dtype: int64


In [ ]:
col_name = []
for feature_batch, label_batch in train_ds.take(1):
    col_name = list(feature_batch.keys())
col_name.remove('age_group')
col_name.remove('race')
col_name.remove('gender')
col_name.remove('religion')

In [ ]:
feature_cols = []
for header in co_name:
    feature_cols.append(feature_column.numeric_column(header))

categorical = feature_column.categorical_column_with_vocabulary_list(
    
)